In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as scipy
from scipy.linalg import toeplitz
from scipy.sparse import csr_matrix, kron, linalg


def blur(N, band=3, sigma=0.7):
    z = np.block([np.exp(-(np.array([range(band)]) ** 2) / (2 * sigma ** 2)), np.zeros((1, N - band))])
    A = toeplitz(z)
    A = csr_matrix(A)
    A = (1 / (2 * scipy.pi * sigma ** 2)) * kron(A, A)

    x = np.zeros((N, N))
    N2 = round(N / 2)
    N3 = round(N / 3)
    N6 = round(N / 6)
    N12 = round(N / 12)

    # Large elipse
    T = np.zeros((N6, N3))
    for i in range(1, N6 + 1):
        for j in range(1, N3 + 1):
            if (i / N6) ** 2 + (j / N3) ** 2 < 1:
                T[i - 1, j - 1] = 1

    T = np.block([np.fliplr(T), T])
    T = np.block([[np.flipud(T)], [T]])
    x[2:2 + 2 * N6, N3 - 1:3 * N3 - 1] = T

    # Small elipse
    T = np.zeros((N6, N3))
    for i in range(1, N6 + 1):
        for j in range(1, N3 + 1):
            if (i / N6) ** 2 + (j / N3) ** 2 < 0.6:
                T[i - 1, j - 1] = 1

    T = np.block([np.fliplr(T), T])
    T = np.block([[np.flipud(T)], [T]])
    x[N6:3 * N6, N3 - 1:3 * N3 - 1] = x[N6:3 * N6, N3 - 1:3 * N3 - 1] + 2 * T
    x[x == 3] = 2 * np.ones((x[x == 3]).shape)

    T = np.triu(np.ones((N3, N3)))
    mT, nT = T.shape
    x[N3 + N12:N3 + N12 + nT, 1:mT + 1] = 3 * T

    T = np.zeros((2 * N6 + 1, 2 * N6 + 1))
    mT, nT = T.shape
    T[N6, :] = np.ones((1, nT))
    T[:, N6] = np.ones((mT))
    x[N2 + N12:N2 + N12 + mT, N2:N2 + nT] = 4 * T

    x = x[:N, :N].reshape(N ** 2, 1)
    b = A @ x

    return A, b, x


In [2]:
import time

def AG(f, gf, L, x0, max_iter):
    start = time.time()
    x0 = np.array(x0)
    grad = gf(x0)
    fs=[f(x0)]
    gs=[grad]
    ts=[0]
    stop = 0
    step = 1
    y = x0
    x = x0

    while stop < max_iter:
        old_x = x
        x = y - (1/L) * grad 
        old_step = step
        step = (1+np.sqrt(1+4*(step**2)))/2
        y = x + ((old_step - 1)/step) * (x - old_x) 
        stop += 1
        grad = gf(y)
        fs.append(f(x))
        gs.append(grad)
        t = time.time() - start
        ts.append(t * 1000)
    return x, fs, gs, ts

def generic_grad(f, gf, x0, L, max_iter):
    start = time.time()
    x0 = np.array(x0)
    grad = gf(x0)
    fs=[f(x0)]
    gs=[grad]
    ts=[0]
    x0 = np.array(x0)
    step = 1/L
    stop = 0
    x = x0

    while stop < max_iter:
        step = 1/L
        x = x - step * grad
        stop += 1
        grad = gf(x)
        fs.append(f(x))
        gs.append(gf(x))
        t = time.time() - start
        ts.append(t * 1000)
    return x, fs, gs, ts

In [3]:
A, b, x = blur(256, 5, 1)
x0 = np.array([0 for j in range(len(b))])
x0 = x0.reshape((len(x0), 1))
pics = []
iters = [1, 10, 100, 1000]


def f(x):
    return (np.linalg.norm(A @ x - b)) ** 2


def gf(x):
    temp = (A @ x) - b
    return 2 * A @ temp


In [4]:
vals = scipy.sparse.linalg.eigsh((A.T@A), return_eigenvectors=False)
L= 2*max(vals)

In [ ]:
for i in iters:
  y, fs, gs, ts = AG(f, gf, L, x0, i)
  y = np.array(y, dtype=np.float128)
  plt.figure(figsize=(6, 6))
  plt.imshow(y.reshape(256, 256), cmap="gray")
  plt.title(f"Output after {i} iterations:")
  plt.show()

In [6]:
x2, fs2, gs2, ts2 = generic_grad(f, gf, x0, L, 1000)

In [ ]:
points = np.arange(len(fs))
points2 = np.arange(len(fs2))

plt.title("fs and ts by iteration")
plt.semilogy(points, np.array(ts), label='ts of Accelerated GD')
plt.semilogy(points2, np.array(ts2), label='ts of GD with constant step')
plt.semilogy(points, np.array(fs), label='fs of Accelerated GD')
plt.semilogy(points2, np.array(fs2), label='fs of GD with constant step')
plt.legend()
plt.show()